In [8]:
import random
import numpy as np
import operator

# Sample data
def target_function(x):
    return x**3 + x**2 + x

X = np.linspace(-1, 1, 20)
y = target_function(X)

# Genes represent indexes to a grammar table
# Grammar table (like a lookup of functions and terminals)
grammar = [
    'add',  # 0
    'sub',  # 1
    'mul',  # 2
    'div',  # 3
    'x',    # 4 terminal
    '1',    # 5 terminal (constant)
    '2',    # 6 terminal (constant)
]

functions = {
    'add': operator.add,
    'sub': operator.sub,
    'mul': operator.mul,
    'div': lambda x, y: x / y if y != 0 else 1,
}

# Genotype: list of integers, each pointing to an element in grammar
genotype_length = 15

# Expression tree node class
class Node:
    def __init__(self, val, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

# Gene expression function: decode genotype to expression tree
def express_genotype(genotype, pos=0):
    if pos >= len(genotype):
        # Return a default terminal node if genotype ended
        return Node('x'), pos
    
    gene = genotype[pos]
    symbol = grammar[gene]
    
    if symbol in functions:  # function node (binary)
        left_node, next_pos = express_genotype(genotype, pos+1)
        right_node, next_pos = express_genotype(genotype, next_pos)
        return Node(symbol, left_node, right_node), next_pos
    else:
        # Terminal node
        return Node(symbol), pos+1


# Evaluate expression tree
def eval_tree(node, x):
    if node is None:
        return 0  # or some neutral value
    if node.val in functions:
        left_val = eval_tree(node.left, x)
        right_val = eval_tree(node.right, x)
        return functions[node.val](left_val, right_val)
    elif node.val == 'x':
        return x
    else:
        return float(node.val)


# Fitness function (MSE)
def fitness(genotype):
    tree, _ = express_genotype(genotype)
    preds = np.array([eval_tree(tree, xi) for xi in X])
    return np.mean((preds - y)**2)

# Mutation
def mutate(genotype, mutation_rate=0.1):
    for i in range(len(genotype)):
        if random.random() < mutation_rate:
            genotype[i] = random.randint(0, len(grammar)-1)
    return genotype

# Crossover (one point)
def crossover(g1, g2):
    point = random.randint(1, len(g1)-1)
    return g1[:point] + g2[point:]

# Initialize population
pop_size = 50
population = [ [random.randint(0,len(grammar)-1) for _ in range(genotype_length)] for _ in range(pop_size)]

# Evolution
generations = 30
for gen in range(generations):
    population.sort(key=fitness)
    print(f"Gen {gen}, Best MSE: {fitness(population[0]):.4f}")
    
    new_pop = population[:10]  # elitism
    while len(new_pop) < pop_size:
        p1 = random.choice(population[:20])
        p2 = random.choice(population[:20])
        child = crossover(p1, p2)
        child = mutate(child)
        new_pop.append(child)
    population = new_pop

# Pretty print best expression
def print_expr(node):
    if node.val in functions:
        return f"({print_expr(node.left)} {node.val} {print_expr(node.right)})"
    else:
        return str(node.val)

best_tree, _ = express_genotype(population[0])
print("Best expression:", print_expr(best_tree))


Gen 0, Best MSE: 0.1975
Gen 1, Best MSE: 0.1975
Gen 2, Best MSE: 0.1975
Gen 3, Best MSE: 0.1975
Gen 4, Best MSE: 0.1975
Gen 5, Best MSE: 0.1975
Gen 6, Best MSE: 0.1975
Gen 7, Best MSE: 0.1975
Gen 8, Best MSE: 0.1975
Gen 9, Best MSE: 0.1234
Gen 10, Best MSE: 0.1234
Gen 11, Best MSE: 0.1234
Gen 12, Best MSE: 0.1234
Gen 13, Best MSE: 0.0724
Gen 14, Best MSE: 0.0724
Gen 15, Best MSE: 0.0724
Gen 16, Best MSE: 0.0724
Gen 17, Best MSE: 0.0724
Gen 18, Best MSE: 0.0724
Gen 19, Best MSE: 0.0724
Gen 20, Best MSE: 0.0724
Gen 21, Best MSE: 0.0724
Gen 22, Best MSE: 0.0724
Gen 23, Best MSE: 0.0724
Gen 24, Best MSE: 0.0724
Gen 25, Best MSE: 0.0724
Gen 26, Best MSE: 0.0724
Gen 27, Best MSE: 0.0724
Gen 28, Best MSE: 0.0724
Gen 29, Best MSE: 0.0724
Best expression: (x mul (2 add x))
